In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
#import matplotlib.patches as patches
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris

#from matplotlib import gridspec
from IPython.display import HTML

plt.xkcd()

HTML("""
<style>
.toggle_box {
    position: fixed;
    align: center;
    padding: 2px;
    top: 17%;
    left: 96%;
    opacity: 0.6;
    z-index: 10000;
}
.toggle_button {
    padding: 1px 3px 0 1px;
}
</style>

<script>
var code_show = true;

function code_toggle() {
  if (code_show) {
    $('div.input').hide('linear');
    $('img#code-toggle')
      .attr('src', 'assets/expand.png')
      .attr('title', 'click to display code cells');
  } else {
    $('div.input').show('linear');
    $('img#code-toggle')
      .attr('src', 'assets/collapse.png')
      .attr('title', 'click to hide code cells');
  }
  code_show = !code_show;
}
</script>

<div class='toggle_box'>
  <button class='toggle_button' onclick='javascript:code_toggle()'>
    <img id='code-toggle' src='assets/collapse.png' title='click to hide code cells'/>
  </button>
</div>""")

In [2]:
# learning rate hyperparameter (step size scaling)
ALPHA = 1e-4

# weight decay hyperparameter (regularization)
LAMBD = 1e-3

# threshold for convergence test
DIFF_THRESHOLD = 1e-5

In [3]:
def sigmoid(z):
    return 1. / (1. + np.exp(-z))

In [4]:
def threshold(a):
    ''' Threshold given value to return 1 if greater than or equal to 0.5,
        or 0 if otherwise.
    '''
    if a >= 0.5:
        return 1
    else:
        return 0

h = np.vectorize(threshold, otypes=[np.dtype('int8')])

In [5]:
def cost(theta, X, y):
    theta = np.matrix(theta)
    
    m = X.shape[0]
    
    z = np.dot(X, theta.T).sum(axis=1)
    a = sigmoid(z)
    y_hat = h(a)
    
    cost_simple = np.add(np.multiply(y, np.log1p(y_hat)),
                         np.multiply((1-y), np.log1p(1-y_hat)))
    
    reg = (LAMBD/2) * np.square(theta[:, 1:X.shape[1]]).sum()
    
    cost = (-1./m) * (cost_simple.sum() + reg)
    
    return cost

In [6]:
def gradient(theta, X, y):
    theta = np.matrix(theta)
    
    m = X.shape[0]
    
    z = np.dot(X, theta.T).sum(axis=1)
    a = sigmoid(z)
    y_hat = h(a)
    err = y_hat - y
    
    derivs = np.multiply(err, X).sum(axis=0).A1 / m
    derivs[1:X.shape[1]] += (LAMBD/m) * theta[:, 1:X.shape[1]].sum()
    
    return derivs

In [7]:
def gradient_descent(theta, X, y):
    iters = 1
    
    m = X.shape[0]
    
    cost_history = []
    
    theta_history = [ np.ones(X.shape[1]) ]
    
    while True:
        theta -= ALPHA * gradient(theta, X, y)
        
        # convergence test
        last_theta = theta_history[-1]
        diffs = np.sum(np.abs(np.subtract(last_theta, theta)) < DIFF_THRESHOLD)
        
        if diffs == X.shape[1]:
            print('... convergence at {} iterations'.format(iters))
            break
        else:
            theta_history.append(np.copy(theta))
            cost_history.append(cost(theta, X, y))
            iters += 1

    return theta, cost_history, theta_history                

In [8]:
iris = load_iris()

In [9]:
np.random.seed(888)

idx_setosa     = np.where(iris.target==0)[0]
idx_versicolor = np.where(iris.target==1)[0]
idx_virginica  = np.where(iris.target==2)[0]

train_setosa     = np.random.choice(idx_setosa, size=40, replace=False)
train_versicolor = np.random.choice(idx_versicolor, size=40, replace=False)
train_virginica  = np.random.choice(idx_virginica, size=40, replace=False)
training_classes = [train_setosa, train_versicolor, train_virginica]

idx_train = np.concatenate(training_classes)
np.random.shuffle(idx_train)

idx_test = np.array([i for i in range(iris.data.shape[0]) if i not in idx_train])
np.random.shuffle(idx_test)

In [10]:
X = iris.data[idx_train, :]
X = np.insert(X, 0, np.ones(idx_train.size), axis=1)

In [11]:
classes = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
classifiers = []

for idx,cls in enumerate(classes):
    print('\nTraining classifier for {}'.format(cls))
    
    iters = 1
    
    y = np.array([e in training_classes[idx] for e in idx_train], dtype=np.int8)
    y = y.reshape((idx_train.size,1))
    
    theta = np.random.normal(scale=1e-3, size=X.shape[1])
    
    final_theta, cost_history, theta_history = gradient_descent(theta, X, y)
    
    classifiers.append(final_theta)

classifiers = np.vstack(classifiers)


Training classifier for Iris-setosa
... convergence at 4 iterations

Training classifier for Iris-versicolor
... convergence at 1625 iterations

Training classifier for Iris-virginica
... convergence at 139 iterations


In [12]:
X_test = iris.data[idx_test, :]
X_test = np.insert(X_test, 0, np.ones(idx_test.size), axis=1)

correct = 0
for i,x in enumerate(X_test):
    z = np.dot(classifiers, x.T)
    g = sigmoid(z)
    y_hat = g.argmax()
    y = iris.target[idx_test[i]]
    
    if y == y_hat:
        correct += 1

    print('> test {0:2d}: classifier predicted {1}, actual class is {2}'.format(i+1,
                                                                                classes[y_hat],
                                                                                classes[y]))

print('\n... Overall classification accuracy: {:.3f}'.format(correct/X_test.shape[0]))

> test  1: classifier predicted Iris-virginica, actual class is Iris-virginica
> test  2: classifier predicted Iris-setosa, actual class is Iris-setosa
> test  3: classifier predicted Iris-virginica, actual class is Iris-virginica
> test  4: classifier predicted Iris-virginica, actual class is Iris-virginica
> test  5: classifier predicted Iris-setosa, actual class is Iris-setosa
> test  6: classifier predicted Iris-setosa, actual class is Iris-setosa
> test  7: classifier predicted Iris-versicolor, actual class is Iris-versicolor
> test  8: classifier predicted Iris-setosa, actual class is Iris-setosa
> test  9: classifier predicted Iris-versicolor, actual class is Iris-versicolor
> test 10: classifier predicted Iris-versicolor, actual class is Iris-versicolor
> test 11: classifier predicted Iris-virginica, actual class is Iris-virginica
> test 12: classifier predicted Iris-virginica, actual class is Iris-versicolor
> test 13: classifier predicted Iris-versicolor, actual class is Iris